Exchange Whales Ratio

거래소 입금액 파생 데이터

상위 10위 거래소 입금액 / 전체 입금액


소스데이터
https://cryptoquant.com/ko/asset/btc/chart/exchange-flows/exchange-inflow-top10?exchange=all_exchange&window=DAY&sma=0&ema=0&priceScale=log&metricScale=linear&chartStyle=line

In [92]:
import os
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.dates as mdates

In [93]:
def ts_to_dt(timestamp):
    datetimeobj = datetime.datetime.fromtimestamp(timestamp/1000, datetime.timezone.utc)
    return datetimeobj


def json_to_df(path, type='non_price'):
     if type != 'price':
          with open(path, 'r') as j:
               contents = json.loads(j.read())

          df = pd.DataFrame(contents['result']['data'])
          df.columns = ['timestamp', 'value']

          df['date'] = df['timestamp'].apply(ts_to_dt).astype(str).apply(lambda x: x[:10])
          df = df.set_index('date')
          df = df[['value']]
          df = df.iloc[:-1]
          return df
     elif type == 'price':
          with open(path, 'r') as j:
               contents = json.loads(j.read())

          df = pd.DataFrame(contents['data'])
          df.columns = ['timestamp', 'value']

          df['date'] = df['timestamp'].apply(ts_to_dt).astype(str).apply(lambda x: x[:10])
          df = df.set_index('date')
          df = df[['value']]
          df = df.iloc[:-1]
          return df

In [94]:
json_to_df(price_list[0], type='price')

,value
date,
2022-03-04,1.348270
2022-03-05,1.358412
2022-03-06,1.303264
2022-03-07,1.272315
2022-03-08,1.297222
...,...
2025-02-27,0.246267
2025-02-28,0.241434
2025-03-01,0.242600


In [95]:
top10_exchange_list = glob('./top10/*.json')
total_deposit_exchange_list = glob('./total_deposit_exchange/*.json')
price_list = glob('./price/*.json')
name_list = []
for total in total_deposit_exchange_list:
    name_list.append(total[25:-5])


In [96]:
json_to_df(price_list[11], type='price')

,value
date,
2022-03-04,39159.177767
2022-03-05,39408.889880
2022-03-06,38418.576257
2022-03-07,38001.870265
2022-03-08,38739.769721
...,...
2025-02-27,84792.374287
2025-02-28,84500.853924
2025-03-01,86103.128568


In [97]:
name_list

['1inch',
 'aave',
 'agld',
 'alcx',
 'alice',
 'amp',
 'ankr',
 'axs',
 'bal',
 'bat',
 'bnt',
 'btc',
 'chz',
 'comp',
 'crv',
 'dodo',
 'dydx',
 'enj',
 'ens',
 'ern',
 'eth',
 'fet',
 'ghst',
 'grt',
 'gtc',
 'hot',
 'ilv',
 'jasmy',
 'knc',
 'link',
 'mana',
 'mask',
 'matic',
 'mkr',
 'nmr',
 'ogn',
 'omg',
 'qnt',
 'sand',
 'shib',
 'skl',
 'slp',
 'snx',
 'storj',
 'sushi',
 'uma',
 'uni',
 'usdc',
 'usdt',
 'yfi',
 'ygg',
 'zrx']

In [98]:
all_parts = []  # Collect all data to concatenate outside the loop
for total, top10, price, name in zip(total_deposit_exchange_list, 
                                     top10_exchange_list, 
                                     price_list, 
                                     name_list):
    total_part_df = json_to_df(total).add_prefix(f'total_{name}_')
    top10_part_df = json_to_df(top10).add_prefix(f'top10_{name}_')
    price_part_df = json_to_df(price, type='price').add_prefix(f'price_{name}_')
    
    combined_df = pd.concat([total_part_df, top10_part_df, price_part_df], axis=1)
    combined_df = combined_df.dropna()
    combined_df[f'deposit_volume_{name}_values'] = combined_df[f'total_{name}_value'] * combined_df[f'price_{name}_value']
    all_parts.append(combined_df)

# Combine all data into one final DataFrame
final_df = pd.concat(all_parts, axis=1, ignore_index=False)
print(final_df)

            total_1inch_value  top10_1inch_value  price_1inch_value  \
date                                                                  
2022-03-04       1.525460e+06       2.005504e+05           1.348270   
2022-03-05       3.592247e+05       8.863093e+04           1.358412   
2022-03-06       3.818926e+05       7.681026e+04           1.303264   
2022-03-07       3.204788e+06       2.865317e+05           1.272315   
2022-03-08       1.072697e+06       1.343846e+05           1.297222   
...                       ...                ...                ...   
2025-02-27       3.803228e+06       3.668520e+05           0.246267   
2025-02-28       5.260905e+06       1.103524e+06           0.241434   
2025-03-01       1.735185e+06       3.048752e+05           0.242600   
2025-03-02       2.645796e+06       1.549856e+05           0.257934   
2025-03-03       3.288554e+06       3.017062e+05           0.224039   

            deposit_volume_1inch_values  total_aave_value  top10_aave_value 

In [99]:
# 'volume'이 포함된 열만 추출
volume_cols = final_df.filter(like='volume')
print(volume_cols)

            deposit_volume_1inch_values  deposit_volume_aave_values  \
date                                                                  
2022-03-04                 2.056733e+06                5.289056e+06   
2022-03-05                 4.879750e+05                3.619432e+06   
2022-03-06                 4.977070e+05                5.080985e+06   
2022-03-07                 4.077499e+06                4.922159e+06   
2022-03-08                 1.391526e+06                7.551331e+06   
...                                 ...                         ...   
2025-02-27                 9.366078e+05                8.355608e+06   
2025-02-28                 1.270160e+06                1.662306e+07   
2025-03-01                 4.209558e+05                6.772633e+06   
2025-03-02                 6.824401e+05                1.494932e+07   
2025-03-03                 7.367630e+05                1.364407e+07   

            deposit_volume_agld_values  deposit_volume_alcx_values  \
date  

In [102]:
volume_cols.mean()

deposit_volume_1inch_values    1.634835e+06
deposit_volume_aave_values     6.900066e+06
deposit_volume_agld_values     8.060136e+05
deposit_volume_alcx_values     4.620880e+05
deposit_volume_alice_values    2.573703e+05
deposit_volume_amp_values      1.430029e+06
deposit_volume_ankr_values     1.525157e+06
deposit_volume_axs_values      2.527329e+06
deposit_volume_bal_values      5.255107e+05
deposit_volume_bat_values      1.116968e+06
deposit_volume_bnt_values      7.267763e+05
deposit_volume_btc_values      1.971504e+09
deposit_volume_chz_values      3.208060e+06
deposit_volume_comp_values     1.614754e+06
deposit_volume_crv_values      4.269359e+06
deposit_volume_dodo_values     1.453681e+05
deposit_volume_dydx_values     1.904498e+06
deposit_volume_enj_values      1.049248e+06
deposit_volume_ens_values      2.736216e+06
deposit_volume_ern_values      2.669174e+05
deposit_volume_eth_values      1.546062e+09
deposit_volume_fet_values      5.276818e+06
deposit_volume_ghst_values     2

In [109]:
# 최댓값을 갖는 열 추출
max_mean_col = volume_cols.mean().idxmax()
max_volume_col = volume_cols[max_mean_col]

print(f"최댓값을 갖는 열: {max_mean_col}")
print(max_volume_col)

최댓값을 갖는 열: deposit_volume_btc_values
date
2022-03-04    1.774846e+09
2022-03-05    7.036626e+08
2022-03-06    5.211025e+08
2022-03-07    1.667952e+09
2022-03-08    1.235010e+09
                  ...     
2025-02-27    4.581654e+09
2025-02-28    4.995210e+09
2025-03-01    1.768501e+09
2025-03-02    2.382306e+09
2025-03-03    4.057208e+09
Name: deposit_volume_btc_values, Length: 1096, dtype: float64
